In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Oct 22 00:13:50 2020

@author: charl
"""
#%%

import os
os.chdir('C:\\Users\\charl\\Desktop\\courses\\AN6001-AI & BIG DATA IN BUSINESS\\pro\\model')


import pandas as pd
import numpy as np
df = pd.read_csv('train.csv').fillna(' ')
df.describe()
import nltk
nltk.download('stopwords')

import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords

from tqdm import tqdm
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import punkt
nltk.download('punkt')
stop_words = stopwords.words('english')

#%%
# string
import string
special = string.punctuation
# remove URL
import re
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub('',text)

df['text']=df['text'].apply(lambda x : remove_URL(x))

## Removing HTML Tags

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

df['text']=df['text'].apply(lambda x: remove_html(x))

## Removing Punctuations

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

df['text']=df['text'].apply(lambda x : remove_punct(x))

text = df['text']

#%%
## create embedding matrix
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
       coefs = np.asarray(values[1:], dtype='float32')
       embeddings_index[word] = coefs
    except ValueError:
       pass
f.close()
print('Found %s word vectors.' % len(embeddings_index))

#%%

def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]# remove stop words
    words = [w for w in words if w.isalpha()] 
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(100)
    return v / np.sqrt((v ** 2).sum())
#%%
# vectorise the text
xtrain_glove = [sent2vec(x) for x in tqdm(text)]
xtrain_glove = np.array(xtrain_glove)
# print(xtrain_glove.shape)
#%%

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(xtrain_glove, df['target'], test_size=0.2, random_state=2020)


#%% GloVe logistic Regression
model1 = LogisticRegression(solver='sag')
model1.fit(X_train, y_train)

y_pred_train=model1.predict(X_train)
y_pred = model1.predict(X_test)
cm_train=confusion_matrix(y_train,y_pred_train)
acc1=(cm_train[0,0]+cm_train[1,1])/sum(sum(cm_train))
print(f'Accuracy of trainset is {acc1}')
cm=confusion_matrix(y_test, y_pred)
import seaborn as sns
import matplotlib.pyplot as plt
acc=round((cm[0,0]+cm[1,1])/sum(sum(cm))*100,2)
recall=round((cm[1,1]/(cm[1,0]+cm[1,1]))*100,2)

plt.figure(figsize=(4.5,3))
sns.heatmap(cm, annot=True, linewidth=0.5, fmt='.0f', cmap='viridis')
plt.title(f'LG GloVe \n Accuracy: {acc}% & Recall Rate: {recall}%')
plt.xlabel('Predict')
plt.ylabel('Actual')
plt.show()


from sklearn import model_selection
cv_score = np.mean(model_selection.cross_val_score(model1, xtrain_glove, df['target'], cv=3, scoring='f1'))
print(f'The Cross-validation accuracy is {round(cv_score*100,2)}%')


#==================================================================================================

#%%

#Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler
# Fitting a simple Naive Bayes on Counts

xtrain_glove_scaled = MinMaxScaler(copy=False).fit_transform(xtrain_glove)

model2 = MultinomialNB()
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(xtrain_glove_scaled, df['target'], test_size=0.2, random_state=2020)


model2.fit(X_train_scaled,y_train)
y_pred_train=model2.predict(X_train_scaled)
y_pred = model2.predict(X_test_scaled)
cm_train=confusion_matrix(y_train,y_pred_train)
acc1=(cm_train[0,0]+cm_train[1,1])/sum(sum(cm_train))
print(f'Accuracy of trainset is {acc1}')
cm=confusion_matrix(y_test, y_pred)
acc=round((cm[0,0]+cm[1,1])/sum(sum(cm))*100,2)
recall=round((cm[1,1]/(cm[1,0]+cm[1,1]))*100,2)

plt.figure(figsize=(4.5,3))
sns.heatmap(cm, annot=True, linewidth=0.5, fmt='.0f', cmap='viridis')
plt.title(f'NB GloVe\n Accuracy: {acc}% & Recall Rate: {recall}%')
plt.xlabel('Predic')
plt.ylabel('Acutal')
plt.show()

from sklearn import model_selection
cv_score = np.mean(model_selection.cross_val_score(model2, xtrain_glove_scaled, df['target'], cv=3, scoring='f1'))
print(f'The Cross-validation accuracy is {round(cv_score*100,2)}%')


#%%